In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Eur_Usd'

In [4]:
def clean_value(val, impact):
    return float(val) * float(impact)

In [5]:
currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
# news = news.loc[(news['Currency_Code'] == currency1) | (news['Currency_Code'] == currency2)]
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.drop(news[(news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Actual_Class'] = news.apply(lambda row: clean_value(row['Actual'], row['Impact']), axis=1)
news['Previous_Class'] = news.apply(lambda row: clean_value(row['Previous'], row['Impact']), axis=1)
base_news, counter_news = news.loc[news['Currency_Code'] == currency1], news.loc[news['Currency_Code'] == currency2]

# Base news
by_date = base_news.groupby('Date')
actual, previous = by_date['Actual_Class'].mean().reset_index(), by_date['Previous_Class'].mean().reset_index()
base_news = base_news.iloc[0:0]
base_news['Date'], base_news[f'Actual_Class_{currency1}'], base_news[f'Previous_Class_{currency1}'] = actual['Date'], actual['Actual_Class'], previous['Previous_Class']

# Counter news
by_date = counter_news.groupby('Date')
actual, previous = by_date['Actual_Class'].mean().reset_index(), by_date['Previous_Class'].mean().reset_index()
counter_news = counter_news.iloc[0:0]
counter_news['Date'], counter_news[f'Actual_Class_{currency2}'], counter_news[f'Previous_Class_{currency2}'] = actual['Date'], actual['Actual_Class'], previous['Previous_Class']

base_news.drop(['Currency_Code', 'Actual', 'Previous', 'Impact', 'Actual_Class', 'Previous_Class'], axis=1, inplace=True)
counter_news.drop(['Currency_Code', 'Actual', 'Previous', 'Impact', 'Actual_Class', 'Previous_Class'], axis=1, inplace=True)

<ipython-input-5-9f2f90e3ff3c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_news['Date'], base_news[f'Actual_Class_{currency1}'], base_news[f'Previous_Class_{currency1}'] = actual['Date'], actual['Actual_Class'], previous['Previous_Class']
<ipython-input-5-9f2f90e3ff3c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counter_news['Date'], counter_news[f'Actual_Class_{currency2}'], counter_news[f'Previous_Class_{currency2}'] = actual['Date'], actual['Actual_Class'], previous['Previous_Class']
/Us

In [6]:
base_news.head()

,Date,Actual_Class_EUR,Previous_Class_EUR
0,2018-01-05 10:00:00+00:00,0.0,0.0
1,2018-01-23 10:00:00+00:00,0.0,0.0
2,2018-01-25 00:00:00+00:00,0.0,0.0
3,2018-01-25 09:00:00+00:00,1.0,1.0
4,2018-01-30 10:00:00+00:00,3.0,0.0


In [7]:
counter_news.head()

,Date,Actual_Class_USD,Previous_Class_USD
0,2018-01-03 15:00:00+00:00,3.0,0.00
1,2018-01-03 19:00:00+00:00,0.0,0.00
2,2018-01-04 13:15:00+00:00,3.0,0.00
3,2018-01-05 13:30:00+00:00,-1.5,0.75
4,2018-01-05 15:00:00+00:00,-3.0,0.00


In [8]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)
df = pd.merge(df, base_news, how='left', on='Date')
df = pd.merge(df, counter_news, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [9]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,Actual_Class_EUR,Previous_Class_EUR,Actual_Class_USD,Previous_Class_USD
0,2021-04-09 06:00:00+00:00,1.18925,1.18985,1.18920,1.18934,1.18939,1.18997,1.18933,1.18946,1.18932,1.18991,1.18926,1.18940,471,-3.0,-3.0,0.0,0.0
1,2021-04-09 06:05:00+00:00,1.18935,1.18958,1.18923,1.18940,1.18947,1.18972,1.18934,1.18952,1.18941,1.18965,1.18929,1.18946,468,-3.0,-3.0,0.0,0.0
2,2021-04-09 06:10:00+00:00,1.18939,1.18940,1.18898,1.18915,1.18951,1.18953,1.18911,1.18927,1.18945,1.18946,1.18904,1.18921,489,-3.0,-3.0,0.0,0.0
3,2021-04-09 06:15:00+00:00,1.18914,1.18919,1.18873,1.18881,1.18926,1.18932,1.18885,1.18894,1.18920,1.18926,1.18879,1.18888,415,-3.0,-3.0,0.0,0.0
4,2021-04-09 06:20:00+00:00,1.18881,1.18914,1.18867,1.18899,1.18892,1.18926,1.18879,1.18913,1.18886,1.18920,1.18873,1.18906,349,-3.0,-3.0,0.0,0.0


In [10]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,Actual_Class_EUR,Previous_Class_EUR,Actual_Class_USD,Previous_Class_USD
73062,2022-04-01 05:35:00+00:00,1.10690,1.10694,1.10668,1.10676,1.10707,1.10710,1.10683,1.10691,1.10698,1.10702,1.10676,1.10684,291,0.0,-3.0,-3.0,3.0
73063,2022-04-01 05:40:00+00:00,1.10675,1.10727,1.10672,1.10720,1.10691,1.10744,1.10689,1.10737,1.10683,1.10735,1.10680,1.10728,400,0.0,-3.0,-3.0,3.0
73064,2022-04-01 05:45:00+00:00,1.10721,1.10722,1.10698,1.10704,1.10737,1.10738,1.10715,1.10721,1.10729,1.10730,1.10707,1.10712,183,0.0,-3.0,-3.0,3.0
73065,2022-04-01 05:50:00+00:00,1.10703,1.10729,1.10700,1.10726,1.10721,1.10744,1.10717,1.10743,1.10712,1.10736,1.10708,1.10734,227,0.0,-3.0,-3.0,3.0
73066,2022-04-01 05:55:00+00:00,1.10727,1.10737,1.10667,1.10673,1.10742,1.10754,1.10683,1.10690,1.10734,1.10745,1.10675,1.10682,439,0.0,-3.0,-3.0,3.0


In [11]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [12]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
n_recent_trades = 2
recent_hours = 1

In [13]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, use_news):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk
    waiting_idx, prev_trade_date = 0, None

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
        actual_base, actual_counter, previous_base, previous_counter = df.loc[df.index[i - 1], [f'Actual_Class_{currency1}', f'Actual_Class_{currency2}', f'Previous_Class_{currency1}', f'Previous_Class_{currency2}']]

        if not use_friday and curr_date.day == 4 and curr_date.hour >= 16:
            continue

        elif prev_trade_date is not None:
            diff = curr_date - prev_trade_date
            n_hours = (diff.total_seconds() / 60) / 60

            if waiting_idx >= n_recent_trades:
                if n_hours >= wait_period:
                    prev_trade_date, waiting_idx = None, 0

                else:
                    continue
        
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        if each_bar:
            buy_signal = all([mid_opens[j] < mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])
            sell_signal = all([mid_opens[j] > mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])

        else:
            buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
            sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_news:
            buy_signal = buy_signal and actual_base > previous_base
            sell_signal = sell_signal and actual_counter > previous_counter

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        placed_trade = False

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

                        placed_trade = True

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

                        placed_trade = True


        if placed_trade and prev_trade_date is not None:
            diff = curr_date - prev_trade_date
            n_hours = (diff.total_seconds() / 60) / 60

            if n_hours <= recent_hours:
                waiting_idx += 1

        closed_trade = False

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None
            closed_trade = True

        if closed_trade:
            prev_trade_date = curr_date

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [14]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10]
each_bar_vals = [False]
n_bars_vals = [3]
pip_movement_vals = [30]
use_pullback_vals = [False]
use_friday_vals = [True]  # TRUE MEANS IT WILL TRADE ALL HOURS ON FRIDAY, FALSE MEANS IT WILL STOP AFTER 9 AM PST
wait_periods = [0]  # THESE ARE IN HOURS
pips_to_risk_vals = ['bars']  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.30]
use_news_vals = [True, False]

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(each_bar_vals) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals) * len(use_friday_vals) * len(wait_periods) * len(pips_to_risk_vals) * len(pullback_percentages) * len(use_news_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for each_bar in each_bar_vals:
            for n_bars in n_bars_vals:
                for pip_movement in pip_movement_vals:
                    for use_pullback in use_pullback_vals:
                        for use_friday in use_friday_vals:
                            for wait_period in wait_periods:
                                for pips_to_risk in pips_to_risk_vals:
                                    for pullback_percentage in pullback_percentages:
                                        for use_news in use_news_vals:
                                            all_combos.append((risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, use_news))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_each_bar_val = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_use_friday_val = None
best_wait_period = None
best_pips_to_risk = None
best_pullback_percentage = None
best_use_news = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, use_news in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, use_friday, wait_period, pips_to_risk, pullback_percentage, use_news)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'each_bar': each_bar, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'use_friday': use_friday, 'wait_period': wait_period, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'use_news': use_news})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_each_bar_val = each_bar
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_use_friday_val = use_friday
        best_wait_period = wait_period
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_use_news = use_news

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 2

639.0210099999987
Num buys: 15
Num sells: 15
Num trades: 30
Num wins: 17
Num losses: 12
Win streak: 5
Loss streak: 3
Avg pips risked: 0.00433966666666666
Remaining runs: 1
Best reward so far: 639.0210099999987

-38.92983999999541
Num buys: 4
Num sells: 4
Num trades: 8
Num wins: 3
Num losses: 5
Win streak: 3
Loss streak: 3
Avg pips risked: 0.0043949999999999545
Remaining runs: 0
Best reward so far: 639.0210099999987



In [16]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best use friday val: ' + str(best_use_friday_val))
print('Best wait period: ' + str(best_wait_period))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best use news val: ' + str(best_use_news))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 639.0210099999987
Best risk/reward ratio: 1.5
Best spread: 0.1
Best each bar val: False
Best n bars val: 3
Best pip movement val: 30
Best use pullback val: False
Best use friday val: True
Best wait period: 0
Best pips to risk: bars
Best pullback percentage: 0.3
Best use news val: False
-----------------------
Top results:
{'reward': 639, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': False, 'use_friday': True, 'wait_period': 0, 'pips_to_risk': 'bars', 'pullback_percentage': 0.3, 'use_news': False}
{'reward': -38, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': False, 'use_friday': True, 'wait_period': 0, 'pips_to_risk': 'bars', 'pullback_percentage': 0.3, 'use_news': True}
